In [278]:
import geopandas as gpd
import numpy as np
import fiona
from shapely.geometry import shape
from shapely.ops import cascaded_union
from shapely.geometry import Polygon

In [279]:
predicted_path = "../data/predicted/predicted_20160103_66979721-be1b-4451-84e0-4a573236defd.geojson"
pred = gpd.read_file(predicted_path)

In [280]:
sum_coords = []
for num, row in pred.iterrows():
    sum_coords.extend(list(row['geometry'].exterior.coords))
    
base_bound = Polygon(sum_coords).bounds


p1 = Polygon(
            [(base_bound[0], base_bound[1]),
             (base_bound[0], base_bound[3]),
             (base_bound[2], base_bound[3]),
             (base_bound[2], base_bound[1])
             ])

In [281]:
gt_path = "../data/clearcuts_kh_2016-12-05/clearcuts_backup_2016-12-05.shp"
gt_poly = gpd.read_file(gt_path)

In [282]:
gt_poly.crs = {'init' :'epsg:4326'}
gt_poly = gt_poly.to_crs({'init': "epsg:32637"})

In [283]:
sum_coords = []
for num, row in gt_poly.iterrows():
    sum_coords.extend(list(row['geometry'].exterior.coords))
    
base_bound = Polygon(sum_coords).bounds

p2 = Polygon(
            [(base_bound[0], base_bound[1]),
             (base_bound[0], base_bound[3]),
             (base_bound[2], base_bound[3]),
             (base_bound[2], base_bound[1])
             ])

In [284]:
p2.exterior.coords.xy

(array('d', [262575.75868537056, 262575.75868537056, 356329.76694924606, 356329.76694924606, 262575.75868537056]),
 array('d', [5478583.44112516, 5581428.357466508, 5581428.357466508, 5478583.44112516, 5478583.44112516]))

In [285]:
p3 = p1.intersection(p2)

In [286]:
united_poly = gpd.GeoDataFrame(geometry=[p3])
united_poly.to_file("../data/predicted/intersection_f1.geojson", driver="GeoJSON")

In [287]:
gt_for_f1 = gt_poly[gt_poly.intersects(p3)]

In [288]:
pred_for_f1 = pred[pred.intersects(p3)]

In [289]:
pred_for_f1.to_file("../data/predicted/res_for_f1.geojson", driver="GeoJSON")

In [290]:
gt_for_f1.to_file("../data/predicted/gt_for_f1.geojson", driver="GeoJSON")

In [291]:
def F1_score(pred_polys, true_polys, threshold):
    """
    Calculates F1 score using shapely polygons.

    :param pred_polys: Predicted polygons.
    :param true_polys: Ground truth polygons.
    :param threshold: IoU threshold.
    :return: F1 score, recall, precision.
    """
    preds = [pred.buffer(0) for pred in pred_polys]
    truths = [true.buffer(0) for true in true_polys]

    if (len(preds) == 0) and (len(truths) == 0):
        precision = 1
        recall = 1
        F1 = 1
        return F1, recall, precision

    true_positives = 0
    false_positives = 0
    false_negatives = 0

    for truth in truths:
        ious = []
        for pred in preds:
            if not truth.intersects(pred):
                ious.append(0)
            else:
                intersection = truth.intersection(pred).area
                union = truth.union(pred).area
                ious.append(intersection / union)

        if len(ious) == 0:
            false_negatives += 1
            continue

        best = max(ious)
        if best >= threshold:
            preds.pop(ious.index(best))
            true_positives += 1
        else:
            false_negatives += 1

    false_positives = len(preds)

    try:
        precision = true_positives / (true_positives + false_positives)
    except:
        print("Precision value undefined, setting to zero")
        precision = 0

    try:
        recall = true_positives / (true_positives + false_negatives)
    except:
        print("Recall value undefined, setting to zero")
        recall = 0

    try:
        F1 = (
            2 * (precision * recall) / (precision + recall)
        )
    except:
        print("F1 value undefined, setting to zero")
        F1 = 0
    return F1, recall, precision

In [292]:
areas = [_poly.area for _poly in gt_for_f1['geometry'].values]

In [293]:
np.max(areas)

610982.4873263139

In [294]:
pred_areas = [_poly.area for _poly in pred_for_f1['geometry'].values]

In [295]:
np.max(pred_areas)

803650.0

In [296]:
len(gt_for_f1['geometry'].values)

974

In [317]:
pred_filtered = pred_for_f1['geometry'].values[np.array(pred_areas) > 10000]

In [318]:
len(pred_filtered)

581

In [319]:
metrics = F1_score(pred_filtered, gt_for_f1['geometry'].values, 0.5)
F1 = metrics[0]

In [320]:

metrics

(0.49260450160771707, 0.3932238193018481, 0.6592082616179001)